![R-Kenntnis](../Pics/header.png "R-Kenntnis")

<div class="alert alert-block alert-info">

* **Titel:** Übungen 04 (Add-On): Pivot-Tabellen mit `dplyr`
* **Autor:** Prof. Dr. Denis Royer
* **Datum:** 08.11.2022 (Version 1.0)

</div>

# BI - R-Kenntnis Lösung zu den Übungen 04 (Add-On): Pivot-Tabellen mit `dplyr`

<div class="alert alert-block alert-warning">
<b>Wichtig:</b> Bevor wir loslegen, müssen wir zunächst einmal ein paar vorbereitende Dinge erledigen:

* Die notwendigen Packages laden (bspw. `tidyverse`)
* Den Datensatz laden (siehe *Data/cube_rohdaten.xlsx*)
* Hinweis: Bei dem Datensartz handelt es sich um den gleich Datensatz, der auch bei der Excel-Übung zum EInsatz kam.
</div>


In [ ]:
library(tidyverse)
library(readxl)
library(DT)

# Hinweise der summarise() Funktion bei der Gruppierung von
# Daten abschaltem - mehr unter:
# https://statisticsglobe.com/dplyr-message-summarise-has-grouped-output-r
options(dplyr.summarise.inform = FALSE)

In [ ]:
# Daten laden 
pipeline_report.raw <- read_excel("../Data/cube_rohdaten.xlsx")

In [ ]:
DT::datatable(pipeline_report.raw)

In [ ]:
# Wie sieht der Datensatz aus?
str(pipeline_report.raw)

# Einfache Pivots erstellen (Aufgabe 2)

In den Folgenden Source-Code Blöcken, werden die Aufgaben aus der Excel Pivot-Tabellen Übung in ***R*** nachgestellt. 

Die Aufgaben sollen als Beispiele für die Nutzung von `dplyr` dienen und die Inhalte aus **Übung 04:** <a href="Uebung 04/R-Kenntnis Uebung 04.ipynb">Datenaufbereitung mit `dplyr`</a> ergänzen.

## Aufgabe 2a

**Frage:**  Wie viele Opportunities werden voraussichtlich im 1. Quartal 2019 in Deutschland abgeschlossen?


In [ ]:
# SCHRITT 1:
# Zunächst geht es darum, die Daten einmal zu filtern, so dass
# nur die Daten mit Export = 0 gentzt werden.
#
# Weiterhin werden die Daten zu den Quartale und Jahre so kombiniert,
# so dass sie in der Form YYYY-Q in eine neue Spalte QuartalJahr
# zum Gruppieren der Daten genutzt werden können. 

aufgabe.2a.step1 <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) 

head(aufgabe.2a.step1)

In [ ]:
# SCHRITT 2:
# Sind die Daten nun aufbereitet, können sie nach QuartalJahr und Phase
# gruppiert (group_by) und die Anzahl der Opportunities dann addiert werden
# (summarise). Bei der Addition werden die leeren Werte (NA) ignoriert
# (Parameter: na.rm = TRUE).

aufgabe.2a.step2 <- aufgabe.2a.step1 %>%
              group_by(QuartalJahr, Phase) %>%
              summarise(Auftraege = sum(AnzahlOpportunities,na.rm = TRUE)) 

# Ausagabe der ersten 6 Werte
head(aufgabe.2a.step2)

In [ ]:
# SCHRITT 3:
# Da die Daten immer noch in ihrer Ursprungsform sind (zeilenweise), 
# müssen sie nun einmal pivotiert werden, so dass die Phasen der Projekte
# und die Qartalssummen gebildet werden können. 
# Die pivot_wider() Funktion übernimmt dies und sortiert die Daten so um, dass in 
# den Zeilen die Projektphasen stehen (values_from = Auftraege) und in den Splaten die
# QuartalJahr Informationen (names_from = QuartalJahr).

aufgabe.2a.step3 <- aufgabe.2a.step2 %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege)

aufgabe.2a.step3

In [ ]:
# SCHRITT 4:
# Im letzten Schritt, sollen die Summen der Zeilen am Ende
# als neue Spalte eimngefügt werden. Hierzu gibt es zwei Wege:

# Weg 1:
# Die Splaten, die summiert werden sollen, werden vorab in einem 
# separaten Vektor gespeichert (cols_to_sum) und dann summiert
# 
# https://data-se.netlify.app/2021/05/27/zeilenweise-operationen-tidyverse-stil/

cols_to_sum <- c("2019-1", "2019-2", "2019-3", "2019-4", "2020-1")
aufgabe.2a.step4 <- aufgabe.2a.step3 %>%rowwise() %>%              
              mutate(summe = sum(across(any_of(cols_to_sum)),na.rm = TRUE))
aufgabe.2a.step4

In [ ]:
# Weg 2:
# Alternativ kann man einfach alle Spalten summieren, die Nummern
# enthalten - dies geht dann wie folgt:

aufgabe.2a.step4 <- aufgabe.2a.step3 %>%rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))
aufgabe.2a.step4

In [ ]:
# Alle Schritte zusammen:

aufgabe.2a <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) %>%
              group_by(QuartalJahr, Phase) %>%
              summarise(Auftraege = sum(AnzahlOpportunities)) %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))

In [ ]:
aufgabe.2a

## Aufgabe 2b

**Frage:** Wie viele Opportunities, bei denen man sich aktuell in Vertragsverhandlungen befindet, werden voraussichtlich im 2. Quartal 2019 abgeschlossen?

In [ ]:
aufgabe.2b <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) %>%
              group_by(QuartalJahr, Land) %>%
              summarise(Auftraege = sum(AnzahlOpportunities)) %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))

aufgabe.2b

## Aufgabe 2c

**Frage:** Für welches Produkt existieren derzeit in den USA die meisten Opportunities?

In [ ]:
aufgabe.2c <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              group_by(Land, Produkt) %>%
              summarise(Auftraege = sum(AnzahlOpportunities)) %>%
              pivot_wider(names_from = Land, values_from = Auftraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))

aufgabe.2c

## Aufgabe 2d

**Frage:** Mit welchem Produkt wird in welcher Branche voraussichtlich am meisten und am wenigsten Umsatz erzielt werden?

In [ ]:
aufgabe.2d <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              group_by(Branche, Produkt) %>%
              summarise(SummeBetraege = sum(Betrag)) %>%
              pivot_wider(names_from = Branche, values_from = SummeBetraege) %>%
              rowwise() %>%              
              mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))

DT::datatable(aufgabe.2d)

## Exkurs: Tabellen aufhübschen mit gt

In [ ]:
library(gt)

# Tabelle mit dem Package gt hübsch machen
myTable <- aufgabe.2d %>%
  gt() %>%
  tab_header(
    title = "Umsatz Dienstleistungen nach Länder",
  ) %>%
  fmt_currency(
    columns = c(2:11),
    currency = "EUR",
  ) 

# Ausgabe als HTML - ist notwendig, damit es in JupyterLab
# auch richtig eingebettet wird
gt:::as.tags.gt_tbl(myTable)

# Kompexere Pivots (Aufgabe 3)

In [ ]:
aufgabe.3a <- pipeline_report.raw %>%
              filter(Export == 0) %>%
              mutate(QuartalJahr = paste(Jahr, Quartal, sep="-")) %>%
              mutate(propabitity = case_when(
                  Phase == '1 Interessiert' ~ 0.05,
                  Phase == '2 Geprüft' ~ 0.10,
                  Phase == '3 Angebotsvorbereitung' ~ 0.20,
                  Phase == '4 Angebot abgegeben' ~ 0.25,
                  Phase == '5 Angebotsverhandlung' ~ 0.50,
                  Phase == '6 Vertragsverhandlung' ~ 0.75,
                  TRUE ~ as.numeric(0))) %>%
              mutate(GewBetrag = propabitity * Betrag) %>%
              group_by(QuartalJahr, Produkt) %>%
              summarise(Auftraege = sum(GewBetrag,na.rm = TRUE)) %>%
              pivot_wider(names_from = QuartalJahr, values_from = Auftraege) %>%
              rowwise() %>%              
               mutate(summe = sum(across(where(is.numeric)),na.rm = TRUE))

In [ ]:
DT::datatable(aufgabe.3a)